Project steps:

In [ ]:
%run dataset_creator.ipynb

In [ ]:
model_name = 'bert' #'roberta'
paragraph_selection_strategy = 'first' #'rand' 'cas' 'first' 'last'
dataset_name = 'asylex-outcome' #'asylex-outcome' 'sentiment1' 'sentiment2'
text_processor = TextProcessor(None)
dh = DatasetHandler(info=None,model=model_name,text_processor=text_processor)
dataset_path, model_path, num_labels = dh.import_paths_and_nlabels(dataset_name,model_name,paragraph_selection_strategy)
#previous_version = dh.read_dataset(dataset_path)

In [ ]:
#experimental_version = dh.create_dataset(dataset_name,paragraph_selection_strategy,subset_dimension=None)
#dh.compare_datasets(previous_version,experimental_version)
dh.write_dataset(dataset_path,experimental_version)

In [ ]:
for model_name in ['bert','roberta']:
    for dataset_name in ['asylex-outcome','asylex-norp']:
        for paragraph_selection_strategy in ['first','last','rand','cas']:
            print(model_name + ' '+dataset_name + ' '+ paragraph_selection_strategy)
            text_processor = TextProcessor(None)
            dh = DatasetHandler(info=None,model=model_name,text_processor=text_processor)
            dataset_path, model_path, num_labels = dh.import_paths_and_nlabels(dataset_name,model_name,paragraph_selection_strategy)
            previous_version = dh.read_dataset(dataset_path)
            experimental_version = dh.create_dataset(dataset_name,paragraph_selection_strategy,subset_dimension=None)
            dh.compare_datasets(previous_version,experimental_version)

In [ ]:
text_processor = TextProcessor(None)
ac = AsyLexCleaner(text_processor=text_processor)
experimental = ac.full_texts_creator()
ac.write_dataset(FULL_TEXTS_REFINED_FILE,experimental)
#original = ac.read_dataset(FULL_TEXTS_REFINED_FILE)
#experimental.equals(original)

Proceed training the model using train_and_test.ipynb

In [ ]:
%run XAI.ipynb

In [ ]:
model_name = 'bert' #'bert'
dataset_name = 'asylex-outcome' #'sentiment1' 'sentiment2' 'asylex-norp'
paragraph_selection_strategy = 'first' #'last' 'cas' 'rand'
train_set,test_set,validation_set,tokenizer,model = Loader.import_dataset_and_model(dataset_name,model_name,paragraph_selection_strategy)
dataset = test_set

In [ ]:
n_embeddings = 6
n_sentences = 2
undecided_threshold = 0.4

my_lime = LIME(['negative','positive'],510,1000,clipped_heatmap=False,analytics=DataAnalysis(method_name='lime',create_plot=True,create_stats=True,relevance_threshold=0.01))

my_shap = SHA(clipped_heatmap=False,analytics=DataAnalysis(method_name='shap',create_plot=True,create_stats=True,relevance_threshold=0.1))
my_doa = DOA(analytics=DataAnalysis(method_name='diff_of_angle_pho',create_plot=True,create_stats=True,relevance_threshold=4.0))
my_rang_csm = rangedCSM(analytics=DataAnalysis(method_name='ranged_csm',create_plot=True,create_stats=True,relevance_threshold=0.35))
method_list = [my_rang_csm]

ac = AnalysisCreator('try1')
_ = model.to('cpu')
embeddings_manager.generate_embeddings(dataset_name,model_name,paragraph_selection_strategy,n_embeddings)
ac.load_embeddings(n_embeddings)
if torch.cuda.is_available():
    model.to('cuda')
ac.create_scores(n_sentences=n_sentences,undecided_threshold=undecided_threshold,method_list=method_list)
ac.create_comparison(method_list)
_ = model.to('cpu')

In [ ]:
# relevance_threshold = 0.3
my_lime = LIME(['negative','positive'],510,1000)
my_shap = SHA(clipped_heatmap=False,analytics=DataAnalysis(create_plot=True,create_stats=True,relevance_threshold=0.1))
my_doa = DOA(analytics=DataAnalysis(create_plot=True,create_stats=True,relevance_threshold=4.0))
my_ned = NED()
my_csm = CSM()
my_new_csm = newCSM()
my_cos_sim = cosine_similarity(analytics=DataAnalysis(create_plot=True,create_stats=True,relevance_threshold=0.35)) #analytics=DataAnalysis(create_plot=True,create_stats=True,relevance_threshold=0.35)

# my_meth = newNED()
# my_emb_class = embedding_classification(analytics=DataAnalysis(create_plot=True,create_stats=True,relevance_threshold=relevance_threshold))